In [ ]:
import sys
sys.path.append("../src")
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt

from nltk.tokenize import word_tokenize
from stylographic_features import get_stylographic_feat
from clustering import KMeansAuthors

from style_point_cloud import style_point_cloud

from distances import mod_hausdorff_dist

from tqdm import tqdm 

tqdm.pandas()

%load_ext autoreload
%autoreload 2

In [ ]:
df = pd.read_json("../data/Reuters.json")

# No need to clean the text here, as it is already in good shape. We only need to lowercase it, since
# we assume it.
df.text = df.text.apply(lambda x: x.lower())

# Generate point cloud for each text. Since this is shorter, we must
# take less tokens
df["point_cloud"] = df.text.apply(lambda text: style_point_cloud(text, window_size=300, window_overlap=100, max_tokens=1_000))

# Remove documents with no points
df = df[df.point_cloud.apply(len) > 0].copy()

In [ ]:
# Create an author label for each point
auth_labels = [[author]*n_chunks for author, n_chunks in 
                zip(df.author, df.point_cloud.apply(len))]

# Flatten
auth_labels = [x for y in auth_labels for x in y]

In [ ]:
X = PCA(n_components=3).fit_transform(np.vstack(df.point_cloud))

cl = KMeansAuthors(n_authors=3)

In [ ]:
cl.fit(X, auth_labels)
cl.best_score

In [ ]:
# Document accuracy
(df.author == cl.predict_document(X, df.point_cloud.apply(len).to_numpy())).sum() / df.author.shape[0]

In [ ]:
predictions = cl.predict(X, author_labels=True)
auth_idx = dict(zip(set(predictions), range(len(set(predictions)))))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
for label in set(predictions): 
    mask_pred = [True if pr == label else False for pr in predictions]
    mask_true = [True if pr == label else False for pr in auth_labels]
    ax1.scatter(X[:, 0][mask_pred], X[:, 2][mask_pred], label=label)
    ax2.scatter(X[:, 0][mask_true], X[:, 2][mask_true], label=label)
    ax1.legend()
    ax2.legend()
    ax1.set_title("KMeans Predicted")
    ax2.set_title("True labels")